<a href="https://colab.research.google.com/github/obeabi/ProjectPortfolio/blob/master/MiniScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# doing necessary imports
import logging
from datetime import datetime
from flask import Flask, render_template, request,jsonify
# from flask_cors import CORS,cross_origin
import requests
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen as uReq, Request
#import pymongo

app = Flask(__name__)  # initialising the flask app with the name 'app'
# Configure logging to write to a text file named 'logfile.txt'
logging.basicConfig(level=logging.INFO, filename='logfile.txt', filemode='a', format='%(asctime)s - %(levelname)s - %(message)s')

lg = logging.getLogger(__name__)
today_date = datetime.today().strftime('%Y-%m-%d')
# Specify your browser's user agent string
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36"

In [11]:
searchString = 'bag'
searchString

'bag'

In [13]:
searchString = searchString.replace(' ', '')
searchString
i = 0
reviews = []
total_pages = 2
url = "https://www.jumia.com.ng"

In [14]:
while i <= total_pages:
  page_url  = "https://www.jumia.com.ng/catalog/?q=" + searchString + "&page="+ str(i)# preparing the URL to search the product on  Jumia
  lg.info("Page %s is available", i)  # Log the message with the page number
  # Create a Request object with the URL and headers
  url_page  = requests.get(page_url, headers={'User-Agent': user_agent})
  # Read the content of the response
  page_html = bs(url_page.content , "html.parser") # parsing the webpage as HTML
  bigboxes  = page_html.findAll("article", {"class": "prd _fb col c-prd"}) # seacrhing for appropriate tag to redirect to the product link

  for box in bigboxes:
    productLink    = url + box.findAll('a')[1]['href']
    prodRes        = requests.get(productLink, headers={'User-Agent': user_agent})  # getting the product page from serve
    prod_html      = bs(prodRes.text, features="html.parser")  # parsing the product page as HTML
    reveiw_section = prod_html.find_all("section", {"class": "card aim -mtm"})

    try:
      reveiw_section = prod_html.find_all("section", {"class": "card aim -mtm"})
      productAllReviewLink = "https://www.jumia.com.ng" + reveiw_section[0].find('a')['href']
      revRes       = requests.get(productAllReviewLink , headers={'User-Agent': user_agent})  # getting the review_url for an individual page
      review_html  = bs(revRes.text,features="html.parser")  # parsing the review page as HTML
      commentboxes = review_html .find_all('article', {'class' : "-pvs -hr _bet"})
    except:
      commentboxes = prod_html .find_all('article', {'class' : "-pvs -hr _bet"})


    # Append today's date to the filename
    filename     = searchString + "_" + today_date + ".csv"
    fw           = open(filename, "a") # creating a local file to save the details
    headers      = "Product, Customer Name, Rating, Heading, Comment \n" # providing the heading of the columns
    fw.write(headers) # writing first the headers to file

    for commentbox in commentboxes:
      try:
        name = commentbox.find_all('div')[2].div.find_all('span')[1].text
        name = name[3:]
      except:
        name = 'No Name'
        lg.info("No name is available")

      try:
        rating =commentbox.div.text
        rating = rating[0]
      except:
        rating ='No Rating'
        lg.info("No rating is available")

      try:
        commentDate = commentbox.find_all('div')[2].div.find_all('span')[0].text
      except:
        commentDate = 'No Comment Date'
        lg.info("No comment date is available")

      try:
        commentHead = commentbox.h3.text.lower()
      except:
        commentHead = 'No Comment Head'
        lg.info("No comment head is available")

      try:
        custComment = commentbox.p.text
      except:
        custComment = 'No Customer Comment'
        lg.info("No customer comment is available")

      fw.write(searchString+","+name.replace(",", ":")+","+rating + "," + commentHead.replace(",", ":") + "," + custComment.replace(",", ":") + "\n")

      mydict = {"Product": searchString, "Name": name, "Rating": rating, "CommentHead": commentHead,"Comment": custComment} # saving that detail to a dictionary
      #x = table.insert_one(mydict) #insertig the dictionary containing the rview comments to the collection
      reviews.append(mydict) #  appending the comments to the review list
  #print(page_url)
  i += 1  # Increment the page number

#lg.error("An error occurred: {}".format(str(e)))

In [15]:
reviews


[{'Product': 'bag',
  'Name': 'Paul',
  'Rating': '5',
  'CommentHead': 'okay',
  'Comment': 'Affordable'},
 {'Product': 'bag',
  'Name': 'Abraham',
  'Rating': '3',
  'CommentHead': 'i like it',
  'Comment': "But the Zip's of the bag are not good at all they just keep on breaking"},
 {'Product': 'bag',
  'Name': 'Deborah Cobham',
  'Rating': '5',
  'CommentHead': 'efficient',
  'Comment': 'Efficient '},
 {'Product': 'bag',
  'Name': 'Yibotemeka',
  'Rating': '2',
  'CommentHead': "i don't like it.",
  'Comment': "Poor quality product. Not durable at all. Everything about it is below expectation and what was advertised what not I got. All the zips are broken and the backstrap are cut. I almost lost my laptop when one strap got cut and the bag fell to the flow from my son's hand.\r\n\r\nHonestly, I will not recommend this bag to anybody."},
 {'Product': 'bag',
  'Name': 'Precious',
  'Rating': '3',
  'CommentHead': 'iteam qaulity',
  'Comment': "The quality isn't what I expected "},
 {'